In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from ast import literal_eval

In [ ]:
data = pd.read_csv("ner_dataset.csv", encoding= 'unicode_escape')
data2 = pd.read_csv("ner.csv", encoding='unicode_escape')

In [ ]:
tags = data.Tag.unique()

In [ ]:
X = list(data2['Sentence'])
Y = list(data2['Tag'])

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

In [ ]:
word2id = tokenizer.word_index

In [ ]:
word2id

In [ ]:
id2word = {}
for key, value in word2id.items():
    id2word[value] = key

In [ ]:
tags2id = {}
for i, tag in enumerate(tags):
    tags2id[tag] = i

In [ ]:
tags2id

In [ ]:
id2tag = {}
for key, value in tags2id.items():
    id2tag[value] = key

In [ ]:
id2tag

In [ ]:
getTags = []
for sentenses in Y:
    getTags.append(literal_eval(sentenses))
allTags = []
for y in getTags:
    eachTag = []
    for tag in y:
        eachTag.append(tags2id[tag]) 
    padded_tags = eachTag + ([tags2id['O']] * (200 - len(eachTag)))
    allTags.append(padded_tags)
allTags

In [ ]:
allIds = pad_sequences(sequences, maxlen=200, padding='post')

In [ ]:
allIds = np.asarray(allIds)
allTags = np.asarray(allTags)

In [ ]:
X_train, X_rem, y_train, y_rem = train_test_split(allIds,allTags, train_size=0.70)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)


train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train = train.shuffle(255).batch(255)
validation = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
validation = validation.batch(255)
test = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test = test.batch(255)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(40000, 200),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=100, activation='softmax', return_sequences=True)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(tags), activation='softmax'))
])
model.summary()
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(train,validation_data=validation,epochs=15)
model.evaluate(test)

In [ ]:
ShuffledS=  X_test[0]
ShuffledS = ShuffledS.reshape((1, 200))
words = []
for word in ShuffledS[ShuffledS > 0].tolist():
    words.append(id2word[word])
raw = ' '.join(words)
print(raw)  
prediction = model.predict(ShuffledS)
prediction = np.argmax(prediction[0], axis=1)
prediction = prediction.tolist()[ : len(words)] 
PredictTags = []
for tag_id in prediction:
    PredictTags.append(id2tag[tag_id])